# Problem #1

The environment in the Nim learning model consists of piles of items. Each pile has the initial state of a certain number of items. Players alternate turns removing items from a single pile of thier choice. The winner is the player that removes the last item. Thus the environment can be seen as a series of states (items in each pile) and actions taken (how many items taken from a pile).

# Problem #2

The agents are the players who take turns removing items from the piles. Each agent's strategy differs.

**Random**: This agent picks a random pile and removes a random number of items from that pile. The probabilites of which follow a unifirm distribution.

**Guru**: This agent always takes the optimal action for the current state. It follows a mathematical formula which guarantees the best decision. The formula is that the guru agent takes items from any pile that will result in the binary sum of the piles' items to equal 0. If this agent goes against another Guru agent, then the one that goes first has a higher chance of winning as it will dictate the game states. This agent utilizes exploitation completely.

**Q-Learner**: This agent utilizes a combination of exploitation and exploration actions. It refers to a Q-table where one axis represents all the possible states, and the other axis all the possible actions. It bases its action on its assigned policy at the time. If it is seeking to exploit, then it will find the state-action pair in the Q-table with the highest value and perform that action. With each action taken, the Q-value will be updated based on hyperparemters of alpha, a learning rate, gamma, discount rate, and a reward value R for the particular action. The exploration policy allows the Q-learner to not always take the action with the highest Q-value for a particular state. By explorating other actions for particular states, it allows the model the chance to learn quicker. The chance of exploration can be tuned with hyperparameter epsilon. As more iterations pass, the model can reduce exploration and take on more a more exploitive approach.

# Problem #3

Regarding the Q-learner model for the Nim game, each action taken has a defined reward. Reward points is added to the Q-value for a state action pair depending on the learning rate and the discount rate in the Bellman equaiton. With a high discount rate (closer to 1), an emphasis is placed on future actions. A discount rate close to zero discourages future actions. This is representative of a form of punishment or prevention of taking certain actions. The learning rate will affect the adoption of new actions. A low learning rate will cause the model to ignore new actions, and stick with the current action for a certain state.

# Problem #4

10 items per pile
3 piles

Take into account an empty pile, so each pile can have 0 through 10 items.

Thus **total states** = 11 x 11 x 11 = **1331**

# Problem #5

Each turn, a player can only choose 1 pile to remove a certain number of items.

The player must remove at least 1 item, out of a max of 10 items in a pile. So for each pile a player has 10 unique actions (remove 1 item, remove 2 items, ...).

For 3 piles, a total of **30 unique actions** are possible

# Problem #6

In [1]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [2]:
def nim_qlearner(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration

    return move, pile  # action

In [3]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side

        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>8s}{wins['A']:5d}  {b:>8s}{wins['B']:5d}")

    return wins['A'], wins['B']

In [4]:
# Play games
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')
play_games(1000, 'Guru', 'Guru') ;

1000 games,   Random  515    Random  485
1000 games,     Guru  998    Random    2
1000 games,   Random   11      Guru  989
1000 games,     Guru  938      Guru   62


In [5]:
qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game

            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [6]:
# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Qlearner', 'Random')
    wins += [a/(a+b)]

1000 games, Qlearner  504    Random  496
1000 games, Qlearner  591    Random  409
1000 games, Qlearner  624    Random  376
1000 games, Qlearner  688    Random  312
1000 games, Qlearner  714    Random  286
1000 games, Qlearner  709    Random  291
1000 games, Qlearner  741    Random  259


In [7]:
print(wins)

[0.504, 0.591, 0.624, 0.688, 0.714, 0.709, 0.741]


In [8]:
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Qlearner', 'Guru')
    wins += [a/(a+b)]

1000 games, Qlearner    3      Guru  997
1000 games, Qlearner    1      Guru  999
1000 games, Qlearner    9      Guru  991
1000 games, Qlearner   17      Guru  983
1000 games, Qlearner   29      Guru  971
1000 games, Qlearner   28      Guru  972
1000 games, Qlearner   23      Guru  977


As expected, the guru out-performs Qlearner.

Below: **Modifications to Q-learner**

In [9]:
# set gamma to 0
qtable, Alpha, Gamma, Reward = None, 1.0, 0.0, 100.0

nim_qlearn(100000)
play_games(1000, 'Qlearner', 'Random')

By setting gamma to 0, the Qlearner improved in performance against the random player from .705 to .801

With gamma=0, it tries to maximize the current reward for an action at a certain state. It does not allow for the propogation of future rewards onto the current state-action pair.

In the above Qlearning model, it is exploring 100% of the time until it finishes the game. By playing a high number of games it allows for the model to approach towards the right Q values.

Thus, the increased performance in gamma=0 can be attributed to reducing the fluctuation in the updates of the Q values. However, this leads to "overfitting" the Q values and mat not lead to a more generalized model.

In [10]:
# observe effect vs guru
play_games(1000, 'Qlearner', 'Guru')

TypeError: 'NoneType' object is not subscriptable

The gamma=0 results in a worse performance against the guru